In [1]:
import pandas as pd
from utils.boxscore import fetch_boxscore_data

In [2]:
"""
I use a dataframe that I created earlier with the script "Game Scrape" to create the URLs needed to collect the data for 
plays.
"""
games = pd.read_csv("games.csv")
games.head()

,season_start_year,away_team,points_away,home_team,points_home,attendance,notes,start_et,datetime,is_regular,game_id
0,2020,Golden State Warriors,99,Brooklyn Nets,125,0.0,NaN,7:00p,2020-12-22,1,020bf616-c1e5-4472-a6bb-f85fa8e503f5
1,2020,Los Angeles Clippers,116,Los Angeles Lakers,109,0.0,NaN,10:00p,2020-12-22,1,62d1e0dc-e7ec-4352-bd23-e2abab4cb759
2,2020,New York Knicks,107,Indiana Pacers,121,0.0,NaN,7:00p,2020-12-23,1,cdcfe2d4-000c-413f-b5de-b27de0548ea7
3,2020,Dallas Mavericks,102,Phoenix Suns,106,0.0,NaN,10:30p,2020-12-23,1,812fac2d-f7b0-41e4-894f-10927f96b6e7
4,2020,Utah Jazz,120,Portland Trail Blazers,100,0.0,NaN,10:00p,2020-12-23,1,855b7fdf-dfde-4b2c-9f7c-24c8ae343592


In [3]:
"""
I add the abbreviation for the home team because I need it for creating the URLs.
"""

teamAbbreviation = {'Dallas Mavericks':'DAL', 'Phoenix Suns':'PHO', 'Boston Celtics':'BOS',
       'Portland Trail Blazers':'POR', 'New Jersey Nets':'NJN', 'Toronto Raptors':'TOR',
       'Los Angeles Lakers':'LAL', 'Utah Jazz':'UTA', 'Philadelphia 76ers':'PHI',
       'New York Knicks':'NYK', 'Minnesota Timberwolves':'MIN', 'Orlando Magic':'ORL',
       'San Antonio Spurs':'SAS', 'Sacramento Kings':'SAC', 'Atlanta Hawks':'ATL',
       'Seattle SuperSonics':'SEA', 'Washington Bullets':'WSB', 'Indiana Pacers':'IND',
       'Los Angeles Clippers':'LAC', 'Miami Heat':'MIA', 'Milwaukee Bucks':'MIL',
       'Charlotte Hornets':'CHO', 'Cleveland Cavaliers':'CLE', 'Houston Rockets':'HOU',
       'Denver Nuggets':'DEN', 'Vancouver Grizzlies':'VAN', 'Golden State Warriors':'GSW',
       'Chicago Bulls':'CHI', 'Detroit Pistons':'DET', 'Washington Wizards':'WAS',
       'Memphis Grizzlies':'MEM', 'New Orleans Hornets':'NOH', 'Charlotte Bobcats':'CHA',
       'New Orleans/Oklahoma City Hornets':'NOK', 'Oklahoma City Thunder':'OKC',
       'Brooklyn Nets':'BRK', 'New Orleans Pelicans':'NOP'}

games['abbreviation'] = games['home_team'].apply(lambda x: teamAbbreviation[x])
games['abbreviation'] = games.apply(
    lambda row: 'CHH' if row.season_start_year < 2014 and row.home_team == 'Charlotte Hornets' else row.abbreviation, 
    axis=1
)

In [4]:
df = fetch_boxscore_data(games, start_index=402, chunk_size=299)


402: Fetching data from https://www.basketball-reference.com/boxscores/202102150SAC.html
403: Fetching data from https://www.basketball-reference.com/boxscores/202102150NYK.html
404: Fetching data from https://www.basketball-reference.com/boxscores/202102150IND.html
405: Fetching data from https://www.basketball-reference.com/boxscores/202102150UTA.html
406: Fetching data from https://www.basketball-reference.com/boxscores/202102160BOS.html
407: Fetching data from https://www.basketball-reference.com/boxscores/202102160MEM.html
408: Fetching data from https://www.basketball-reference.com/boxscores/202102160MIL.html
409: Fetching data from https://www.basketball-reference.com/boxscores/202102160MIN.html
410: Fetching data from https://www.basketball-reference.com/boxscores/202102160OKC.html
411: Fetching data from https://www.basketball-reference.com/boxscores/202102160PHO.html
412: Fetching data from https://www.basketball-reference.com/boxscores/202102170GSW.html
413: Fetching data fr

In [ ]:
# Initialize a new column with 0, assuming all players are not starters initially
df['is_starter'] = 0

# Set the 'is_starter' value to 1 for the first player in the DataFrame
df.loc[0, 'is_starter'] = 1

# Find the indices of the rows where player_name is 'Team Totals' and set 'is_starter' to 1
df.loc[df['player_name'] == 'Team Totals', 'is_starter'] = 1

# The rows between 'Reserves' and 'Team Totals' are considered reserves.
reserve_indices = df[df['player_name'] == 'Reserves'].index
team_total_indices = df[df['player_name'] == 'Team Totals'].index

# Loop through and set 'is_starter' to 0 for reserves
for reserve_idx, team_total_idx in zip(reserve_indices, team_total_indices):
    df.loc[reserve_idx + 1:team_total_idx - 1, 'is_starter'] = 0

In [ ]:
# Cut values that do not include data on players
df = df[(df.player_name!='Reserves')&(df.player_name!='Team Totals')]

In [ ]:
df.to_csv("boxscore_1701-2200.csv", index=False)